# VacationPy

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from dotenv import load_dotenv
import os
#adds variable from environments - need to pip install python-dotenv
load_dotenv()

###

# Establish Key Variables
g_key = os.environ.get("g_key")  # Comment out this line and replace value of g_key with a valid API key

### Store WeatherPy results into DataFrame
* Load the csv exported from WeatherPy to a DataFrame

In [3]:
#imports CSV from WeatherPy output
inputFile = "assets/data/cities.csv"
ReadFile = pd.read_csv(inputFile)
#stores file into dataframe
df = pd.DataFrame(ReadFile)
#removes column added by csv input
del df["Unnamed: 0"]
df.head()

,city,lat,lng,max_temp,humidity,cloudiness,wind_speed,country,date
0,Srednekolymsk,67.45,153.68,12.61,91,21,5.23,RU,1603313661
1,Montevideo,-34.83,-56.17,60.39,67,36,17.22,UY,1603313817
2,Hilo,19.73,-155.09,79.99,51,1,3.36,US,1603313870
3,Ixtapa,20.70,-105.20,88.90,79,20,11.41,MX,1603313353
4,Pascagoula,30.37,-88.56,86.65,52,75,10.29,US,1603313872


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#configures gmaps and sets up api key
gmaps.configure(api_key=g_key)
#establishes figure to add layers to
fig = gmaps.figure()
#pulls location data from dataframe
locations = df[["lat","lng"]]
#pulls humidity data from dataframe
weightHumidity = df["humidity"]
#creates heatmap layer with prior input
heatMap=gmaps.heatmap_layer(locations, weights=weightHumidity,dissipating=False, max_intensity=df.humidity.max(),point_radius=5)
#plots heatlayer on map
fig.add_layer(heatMap)
#shows map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#selects an ideal sample of locations with clouds and wind, but still not freezing - I want to camp in a storm
ideal_df = df.loc[(df.max_temp < 85) & (df.max_temp > 45) & (df.wind_speed > 20) & (df.cloudiness > 50)]

### Hotel Map
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#set new dataframe equal to prior with a few alterations for the info_box_template code below
hotel_df = ideal_df
hotel_df = hotel_df.rename(columns = {"city":"City","country":"Country"})

In [7]:
# find the closest hotel near each city
#initialize google url
google_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#list will hold dataframe values
hotels = []
#loops through all rows and searches coordinates through api to find hotel nearby
for index, row in hotel_df.iterrows():
        #setting search parameters and api_key
    params= {
    "keyword":"hotel",
    "radius":5000,
    "key":g_key,
    "types":"lodging"
    }
    #merge coordinates into string for url to accept
    params["location"] = str(row["lat"]) + ", " + str(row["lng"])
    #get json file from URL
    data = requests.get(google_url,params=params).json()
    try:
        #if there is data, add it to the hotel list
        hotels.append(data["results"][0]["name"])
    except:
        #if error, add blank space, alert user, and move on
        hotels.append("")
        print("We're sorry, there are no hotels in a 5000 meter radius of " + row["City"])
#puts data into last series of dataframe and prints
hotel_df["Hotel Name"] = hotels

In [8]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [9]:
#establishes figure to add layers to
fig2 = gmaps.figure()
#adds markers for the lat/long we searched
markerMap = gmaps.marker_layer(locations)
#adds layer to figure
fig2.add_layer(markerMap)
# Creates layer to show name, city, and country of locations when hovering. Set formatting to blue/yellow
info_layer = gmaps.symbol_layer(
    locations, fill_color="yellow",
    stroke_color='blue', scale=5,
    info_box_content=hotel_info
)
#adds last layer to figure
fig2.add_layer(info_layer)
#shows figure
fig2

Figure(layout=FigureLayout(height='420px'))